In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from imp import reload
import vnn2
reload(vnn2)

<module 'vnn2' from '/Users/davidclark/Projects/VectorizedNets/vnn2.py'>

In [15]:
lc = vnn2.Local2d(12, 12, 5, 32, 32, stride=1, padding=2, bias=True)

In [16]:
x = torch.randn(128, 12, 32, 32)
out = lc(x)
out.shape

torch.Size([128, 12, 32, 32])

In [17]:
loss = out.sum()

In [18]:
loss.backward()

In [19]:
lc.weight.grad.shape

torch.Size([12, 32, 32, 12, 5, 5])